# 볼린저 밴드
1. 파일 로드 
2. 결측치와 이상치를 제거 
3. 시간, 종가만의 데이터프레임을 생성
4. 이동 평균선 : 데이터 20개의 평균을 구해서 새로운 파생변수 대입
5. 상단 밴드 : 이동 평균선 + (2 * 데이터 20개의 표준편차)
6. 하단 밴드 : 이동 평균선 - (2 * 데이터 20개의 표준편차)
7. 구매 상태를 확인하는 파생변수 생성
8. 구매 상태 입력
9. 수익율 계산

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../csv/GM.csv", index_col='Date')
df.head()

In [ ]:
df = df.loc[~df.isin((np.nan, np.inf, -np.inf)).any(axis='columns')]
df

In [ ]:
# 시간과 종가만의 데이터프레임 생성
price_df = df[['Adj Close']]
price_df.head(1)

In [ ]:
price_df['center'] = 0

In [ ]:
# 이동 평균선 : 종가 데이터 20개의 평균값

price_df['center'] = price_df['Adj Close'].rolling(20).mean()

In [ ]:
# ub, lb 두개의 파생변수 생성
# 상단밴드 생성
price_df['ub'] = price_df['center'] + ( 2 * price_df['Adj Close'].rolling(20).std() )
# 하단밴드 생성
price_df['lb'] = price_df['center'] - ( 2 * price_df['Adj Close'].rolling(20).std() )

In [ ]:
price_df.tail()

In [ ]:
test_df = price_df.tail(100)

In [ ]:
test_df.drop('Adj Close', axis=1).plot()

In [ ]:
start = "2011-01-02"
price_df2 = price_df.loc[start:]
price_df2.head(2)

In [ ]:
# 거래 내역이라는 파생변수 
price_df2['trade'] = ""
price_df2.head(2)

### 구매 내역 추가
- 조건
    1. 상단 밴드보다 종가가 높은 경우
        - 현재 구매 상태이면
            - 매도 trade = ""
        - 구매 상태가 아니면
            - 아무 행동도 하지 않는다. trade = ""
    2. 하단 밴드보다 종가가 낮은 경우
        - 현재 구매 상태이면
            - 아무 행동도 하지 않는다. trade = 'buy'
        - 구매 상태가 아니면
            - 매수 trade = 'buy'
    3. 하단 밴드와 상단 밴드 사이에 종가가 존재하는 경우
        - 현재 구매 상태이면
            - 아무 행동도 하지 않는다. trade = 'buy'
        - 구매 상태가 아니면
            - 아무 행동도 하지 않는다. trade = ''

In [ ]:
for i in price_df2.index:
    # 상단 밴드보다 종가가 높은 경우
    if price_df2.loc[i, 'Adj Close'] > price_df2.loc[i, 'ub']:
        # 현재 구매 상태이면
        if price_df2.shift(1).loc[i, 'trade'] == 'buy':
            # 매도
            price_df2.loc[i, 'trade'] = ''
        else:
            price_df2.loc[i, 'trade'] = ''
    # 하단 밴드보다 종가가 낮은 경우
    elif price_df2.loc[i, 'Adj Close'] < price_df2.loc[i, 'lb']:
        # 현재 구매 상태이면
        if price_df2.shift(1).loc[i, 'trade'] == 'buy':
            # 구매 상태를 유지
            price_df2.loc[i, 'trade'] = 'buy'
        else:
            # 매수
            price_df2.loc[i, 'trade'] = 'buy' 
    else:
        # 현재 구매 상태이면
        if price_df2.shift(1).loc[i, 'trade'] == 'buy':
            # 구매 상태를 유지
            price_df2.loc[i, 'trade'] = 'buy'
        else:
            price_df2.loc[i, 'trade'] = ''

In [ ]:
price_df2['trade'].value_counts()

### 수익율 계산
1. 구매한 날의 종가
    - 전날의 trade 항목의 값이 '' 이고 현재의 trade가 'buy'인 날의 종가
2. 판매한 날의 종가
    - 전날의 trade 가 'buy' 현재의 trade가 ''인 날의 종가
3. 수익율 계산
    - (판매 가격 - 구매 가격) / 구매 가격 + 1
4. 구매 가격과 판매 가격을 초기화
5. 여러개의 수익율 발생
6. 여러개의 수익율로 누적 수익율 계산

In [ ]:
rtn = 1.0
price_df2['return'] = 1
buy = 0.0
sell = 0.0

for i in price_df2.index:
    # 구매가를 출력
    if (price_df2.shift(1).loc[i, 'trade'] == '') and \
        (price_df2.loc[i, 'trade'] == 'buy'):
        buy = price_df2.loc[i, 'Adj Close']
        print('진입일 :', i, '구매 가격 :', buy)
    # 판매가를 출력
    elif (price_df2.shift(1).loc[i, 'trade'] == 'buy') and \
        (price_df2.loc[i, 'trade'] == ''):
        sell = price_df2.loc[i, 'Adj Close']
        rtn = (sell - buy) / buy + 1
        price_df2.loc[i, 'return'] = rtn
        print('판매일 :', i, '판매 가격 :', sell, '수익율 :', rtn)

    # 구매가, 판매가를 초기화
    if price_df2.loc[i, 'trade'] == '':
        buy = 0.0
        sell = 0.0

In [ ]:
#  누적 수익율 
acc_rtn = 1.0

for i in price_df2.index:
    rtn = price_df2.loc[i, 'return']
    acc_rtn *= rtn
    price_df2.loc[i, 'acc_rtn'] = acc_rtn

print('누적 수익율 :', acc_rtn)

In [ ]:
price_df2.iloc[400:500]